Starting with our mystery fasta file containing an unknown, interesting sequences we found in our studies, let's use Biopython's http BLAST methods to retrieve a BLAST query result.  We can save the result to an XML file the default output method of the BLAST search.

In [1]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO

In [2]:
# fp = "./files/dna.example.fasta"
fp = "./files/dna2.fasta"
records = list(SeqIO.parse(fp, "fasta"))

# How many records are in a file?
print(len(records))

f = open(fp).read()
print(f.count('>'))

18
18


In [3]:
"""
'annotations',
 'dbxrefs',
 'description',
 'features',
 'format',
 'id',
 'letter_annotations',
 'lower',
 'name',
 'reverse_complement',
 'seq',
 'translate',
 'upper'
 """


l_seq = 0
s_seq = 0
for record in records:
    seq_len = len(record.seq)
    print(seq_len)
    
    # Check if its the longest sequence
    if seq_len > l_seq:
        l_seq = seq_len
        
    # Check if its the shortest sequence
    if seq_len < s_seq or s_seq == 0:
        s_seq = seq_len

# What is the longest sequence?
print(l_seq)

# What is the shortest sequence?
print(s_seq)

# Is there more than one longest or shortest sequence?

# What are their identifiers?



4635
1151
4894
3511
4076
2867
442
890
967
4338
1352
4564
4804
964
2095
1432
115
2646
4894
115


In [4]:
"""
In molecular biology, a reading frame is a way of dividing the DNA sequence of nucleotides into a set of 
- consecutive, 
- non-overlapping triplets (or codons). 

Depending on where we start, there are six possible reading frames: 
- three in the forward (5' to 3') direction and 
- three in the reverse (3' to 5'). 

For instance, the three possible forward reading frames for the sequence AGGTGACACCGCAAGCCTTATATTAGC are:

AGG TGA CAC CGC AAG CCT TAT ATT AGC

A GGT GAC ACC GCA AGC CTT ATA TTA GC

AG GTG ACA CCG CAA GCC TTA TAT TAG C

These are called reading frames 1, 2, and 3 respectively. 
An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. 
It starts with a start codon (ATG), and ends with a stop codon (TAA, TAG or TGA). 
For instance, ATGAAATAG is an ORF of length 9.

Given an input reading frame on the forward strand (1, 2, or 3) your program should be able to 
identify all ORFs present in each sequence of the FASTA file, and answer the following questions: 
- What is the length of the longest ORF in the file? 
- What is the identifier of the sequence containing the longest ORF? 
- For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? What is the starting position of the longest ORF in the sequence that contains it? The position should indicate the character number in the sequence. For instance, the following ORF in reading frame 1:

>sequence1

ATGCCCTAG

starts at position 1.

Note that because the following sequence:

>sequence2

ATGAAAAAA

does not have any stop codon in reading frame 1, we do not consider it to be an ORF in reading frame 1."""

start_codon = "ATG"
end_codons = ["TAA", "TAG", "TGA"]
longest_orf = 0  # Longest ORF record
longest_orf_sp = 0  # Longest ORF record, starting position

# Loop through all the records
for i, record in enumerate(records):
#     record = record.reverse_complement()
#     print(i)
    
    # Build a reading frame 2
    n = 0 # For frame 2 (offset by 1 element)
    rf_2 = [str(record.seq[i: i + 3]) for i in range(n, len(record), 3)]
    
    start_pos = 0
    start_flag = False
        
    # Loop through the sequence chunks
    for j, chunk in enumerate(rf_2):
        # If a start codon is found, log pos
        if chunk == start_codon and not start_flag:
            start_pos = j
            start_flag = True
            
        # if an end condon is found, log pos
        if chunk in end_codons and start_flag:
            orf = (j - start_pos + 1) * 3
            if orf > longest_orf:
                longest_orf = orf
                longest_orf_sp = start_pos
            start_flag = False
                
print(longest_orf)
print(longest_orf_sp * 3)

2394
384


In [ ]:
"""
A repeat is a substring of a DNA sequence that occurs in multiple copies (more than one) somewhere in the sequence. Although repeats can occur on both the forward and reverse strands of the DNA sequence, we will only consider repeats on the forward strand here. Also we will allow repeats to overlap themselves. For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), and once at position 3. Given a length n, your program should be able to identify all repeats of length n in all sequences in the FASTA file. Your program should also determine how many times each repeat occurs in the file, and which is the most frequent repeat of a given length.
"""



In [ ]:
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)

with open("blast_output.xml", "w") as output_xml:
    output_xml.write(result_handle.read())
result_handle.close()

Now that the result is saved to file, we do not need to re-run this cell, and thus avoid having to re-run the BLAST query.  With the query result saved to file, we can load the result into a BLAST record object (a type of [python class](https://docs.python.org/3/tutorial/classes.html)) and use Biopython's NCBIXML to return the BLAST record object.  The BLAST record object essentially contains all the information you might need in a BLAST record.  You can view the raw [BLAST record code](http://biopython.org/DIST/docs/api/Bio.Blast.Record-module.html) to see what data are stored in the object.  We can view the alignments, alignment scores, expectation values, and other parameters that are stored in the hsps objects within the BLAST record class.  Because there are typically many alignments for each hit from BLAST, we will just examine the alignments from our top hit.

In [ ]:
result_handle = open("blast_output.xml")
from Bio.Blast import NCBIXML
blast_records = NCBIXML.read(result_handle)
for alignment in blast_records.alignments:
    for hsp in alignment.hsps:
        print(hsp)

Let's now look at what the top 10 hits we get from NCBI's Nucleotide (nt) database.  We will view the decriptions.

In [ ]:
result_handle = open("blast_output.xml")
blast_records = NCBIXML.read(result_handle)
counter = 0
for description in blast_records.descriptions:
    print(description)
    counter += 1
    if counter == 10:
        break

Our top hit is a mRNA transcript record from an organism named Drosophila yakuba.  While outside the scope of this tutorial, one could get more information on this record is by using efetch from the nucleotide database of NCBI via Biopython's Entrez module, as shown below.  [EFetch](https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch) is part of [NCBI's E-Utilities Suite](https://www.ncbi.nlm.nih.gov/books/NBK25497/), which provides programmatic access to records and information from NCBI's Entrez databases.

In [ ]:
from Bio import Entrez
Entrez.email = "anon@example.com"     # Always tell NCBI who you are
top_hit_record = Entrez.efetch(db="nucleotide", id="969472224", rettype="gb", retmode="text")
print(top_hit_record.read())

Returning to our BLAST query, let's explore a little further different parameters for BLAST.  We can first print out the parameters for the qblast function using inspect.signature(function_name).  One could also print the more verbose help(function_name).

In [ ]:
import inspect
print(inspect.signature(NCBIWWW.qblast))

There are many parameters for this function.  Many of them need not be adjusted in most cases.  For simplicity, let's focus on the four parameters below.  First, we create a python dictionary for a few different parameters we would like to adjust to ultimately compare outputs.  We can start by adjusting the database we want to query against.  We will store the output of each blast XML separately should we choose to access the data at a later point.  As we loop through each blast query for the different programs, we will collect 

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib
blast_params = {'program': 'blastn', 'database': 'nt', 'sequence': fasta_string, 'expect': 10.0}
blast_params['database'] = ['nt', 'refseq_representative_genomes']
print_data = pd.DataFrame()
for database in blast_params['database']:
    db_values = {}
    #result = NCBIWWW.qblast(blast_params['program'], database, blast_params['sequence'], expect=blast_params['expect'])
    file_name = "blast_output_" + database + ".xml"
    #with open(file_name, "w") as output_xml:
    #    output_xml.write(result.read())
    result.close()
    result_input = open(file_name)
    blast_records = NCBIXML.read(result_input)
    for description in blast_records.descriptions:
        if 'score' in db_values:
            db_values['score'].append(description.score)
        else:
            db_values['score'] = [description.score]
        if 'e-value' in db_values:
            db_values['e-value'].append(description.e)
        else:
            db_values['e-value'] = [description.e]
    df = pd.DataFrame.from_dict(db_values)
    df['database'] = database[0:6] # we simply limit the name to the first 6 characters for easier viewing
    frames = [print_data, df]
    print_data = pd.concat(frames, ignore_index=True)
print_data.boxplot('score', by='database')
print_data.boxplot('e-value', by='database')

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib
blast_params = {'program': 'blastn', 'database': 'nt', 'sequence': fasta_string, 'expect': 10.0}
blast_params['database'] = ['nt', 'refseq_representative_genomes']
print_data = pd.DataFrame()
for database in blast_params['database']:
    db_values = {}
    #result = NCBIWWW.qblast(blast_params['program'], database, blast_params['sequence'], expect=blast_params['expect'])
    file_name = "blast_output_" + database + ".xml"
    #with open(file_name, "w") as output_xml:
    #    output_xml.write(result.read())
    #result.close()
    result_input = open(file_name)
    blast_records = NCBIXML.read(result_input)
    for description in blast_records.descriptions:
        if 'score' in db_values:
            db_values['score'].append(description.score)
        else:
            db_values['score'] = [description.score]
        if 'e-value' in db_values:
            db_values['e-value'].append(description.e)
        else:
            db_values['e-value'] = [description.e]
    df = pd.DataFrame.from_dict(db_values)
    df['database'] = database[0:6] # we simply limit the name to the first 6 characters for easier viewing
    frames = [print_data, df]
    print_data = pd.concat(frames, ignore_index=True)
print_data.boxplot('score', by='database')
print_data.boxplot('e-value', by='database')

Based our initial results, our top hit for a genome in NCBI Entrez has an accession number of AE014135.4.  We can use this entry to explore genes that are present on this model organism to see what potential biological activities our DNA sequence might be involved with.  Before we move on to notebook 2, we need to find out where on this model genomic sequence our test sequence lies (we would not want to look at ALL of the genes on this chromosome, as our sequence only applies to a small portion of that chromosome).

In [ ]:
from Bio import SearchIO
qresults = next(SearchIO.parse('blast_output.xml', 'blast-xml'))
print(qresults[3]) # the 4th target is out first genomic hit